In [ ]:
from google.colab import drive
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from sklearn import model_selection
from nltk.tokenize import word_tokenize
import nltk
from sklearn.metrics import accuracy_score
nltk.download('punkt')
import codecs

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pprint # for pretty printing our device stats

if 'COLAB_TPU_ADDR' not in os.environ:
    print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print ('TPU address is', tpu_address)

    with tf.compat.v1.Session(tpu_address) as session:
      devices = session.list_devices()

    print('TPU devices:')
    pprint.pprint(devices)

TPU address is grpc://10.76.186.210:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 27977471625426638),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 8589934592, -4948639327698615341),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 9124782786937714842),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, -2338562851466915965),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, -7036378374104742356),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 4755702993681094860),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 1295291329063128586),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 5786615148161536655),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, -

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.76.186.210:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.76.186.210:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
!wget http://nlp.stanford.edu/data/glove.twitter.27B.zip
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip

--2021-05-09 12:58:17--  http://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.twitter.27B.zip [following]
--2021-05-09 12:58:17--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip [following]
--2021-05-09 12:58:17--  http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408563 (1.4G) [appli

In [ ]:
!unzip glove.twitter.27B.zip
# !unzip wiki-news-300d-1M.vec.zip

Archive:  glove.twitter.27B.zip
  inflating: glove.twitter.27B.25d.txt  
  inflating: glove.twitter.27B.50d.txt  
  inflating: glove.twitter.27B.100d.txt  
  inflating: glove.twitter.27B.200d.txt  


In [ ]:
class word_emb_model:
  def __init__(self,path):
      df = pd.read_csv(path)
      # self.encoder = LabelEncoder()
      self.models_glove = {}
      self.history_glove = {}
      
      self.models_fasttext = {}
      self.history_fasttext = {}
      # self.accuracy_glove ={}
      df['text'] = df['text'].apply(lambda x: str(x))
      self.dfs_train = df[df['kfold']!=0]
      self.dfs_valid = df[df['kfold']==0]
      self.read_glove()
      self.data_per_fold_glove()
      # self.read_fasttext()
      # self.data_per_fold_fasttext()

  def read_glove(self):
      embeddings_index = {}
      f = open('glove.twitter.27B.200d.txt')
      for line in f:
          values = line.split()
          word = values[0]
          coefs = np.asarray(values[1:], dtype='float32')
          embeddings_index[word] = coefs
      f.close()

      print('Found %s word vectors.' % len(embeddings_index))
      self.glove_embeddings_index = embeddings_index

  def read_fasttext(self):
      embeddings_index = {}
      f = codecs.open('wiki-news-300d-1M.vec', encoding='utf-8')
      for line in f:
          values = line.rstrip().rsplit(' ')
          word = values[0]
          coefs = np.asarray(values[1:], dtype='float32')
          embeddings_index[word] = coefs
      f.close()
      print('found %s word vectors' % len(embeddings_index))
      self.fasttext_embeddings_index = embeddings_index

  def create_glove_embedding_matrix(self,embed_dim):
      train = self.dfs_train
      test = self.dfs_valid
      x_train = train.text.values
      y_train = train.lbl_num.values
      x_test = test.text.values
      y_test = test.lbl_num.values
      word_tokenizer = Tokenizer()
      word_tokenizer.fit_on_texts(x_train)
      vocab_length = len(word_tokenizer.word_index) + 1
      x_train_emb = word_tokenizer.texts_to_sequences(x_train)
      x_test_emb = word_tokenizer.texts_to_sequences(x_test)
      
      word_count = lambda sentence: len(word_tokenize(sentence))
      longest_sentence = max(np.concatenate((x_train,x_test),axis = 0), key=word_count)
      length_long_sentence = len(word_tokenize(longest_sentence))
      # print('length_long_sentence: '+str(length_long_sentence))
      self.glove_wt = word_tokenizer
      self.lls = length_long_sentence
      x_train_padded = pad_sequences(x_train_emb, length_long_sentence, padding='post',truncating = 'post')
      x_test_padded = pad_sequences(x_test_emb, length_long_sentence, padding='post',truncating = 'post')

      # encoded_Y_tr = self.encoder.fit_transform(y_train)
      # encoded_Y_ts = self.encoder.fit_transform(y_test)
      dummy_y_tr = np_utils.to_categorical(y_train)
      dummy_y_ts = np_utils.to_categorical(y_test)

      embedding_matrix = np.zeros((vocab_length, embed_dim))
      for word, index in word_tokenizer.word_index.items():
          embedding_vector = self.glove_embeddings_index.get(word)
          if embedding_vector is not None:
              embedding_matrix[index] = embedding_vector

      # print(x_train_padded)
      # print(x_test_padded)
      return ((tf.convert_to_tensor(x_train_padded,np.int32),tf.convert_to_tensor(dummy_y_tr,np.float32)),(tf.convert_to_tensor(x_test_padded,np.int32),tf.convert_to_tensor(dummy_y_ts,np.float32)),(vocab_length,embedding_matrix,embed_dim,length_long_sentence))

  def create_fasttext_embedding_matrix(self,embed_dim):
      train = self.dfs_train
      test = self.dfs_valid
      x_train = train.text.values
      y_train = train.lbl_num.values
      x_test = test.text.values
      y_test = test.lbl_num.values
      word_tokenizer = Tokenizer(num_words = 50000,oov_token="<OOV>")
      word_tokenizer.fit_on_texts(x_train)
      vocab_length = len(word_tokenizer.word_index) + 1
      x_train_emb = word_tokenizer.texts_to_sequences(x_train)
      x_test_emb = word_tokenizer.texts_to_sequences(x_test)
      
      word_count = lambda sentence: len(word_tokenize(sentence))
      longest_sentence = max(np.concatenate((x_train,x_test),axis = 0), key=word_count)
      length_long_sentence = len(word_tokenize(longest_sentence))

      self.fasttext_wt = word_tokenizer
      self.lls = length_long_sentence
      x_train_padded = pad_sequences(x_train_emb, length_long_sentence, padding='post',truncating = 'post')
      x_test_padded = pad_sequences(x_test_emb, length_long_sentence, padding='post',truncating = 'post')

      # encoded_Y_tr = self.encoder.fit_transform(y_train)
      # encoded_Y_ts = self.encoder.fit_transform(y_test)
      dummy_y_tr = np_utils.to_categorical(y_train)
      dummy_y_ts = np_utils.to_categorical(y_test)
      # self.df_labels = pd.DataFrame.from_dict({'y_train':y_train,'y_test':y_test,'dummy_tr':dummy_y_tr,'dummy_y_ts':dummy_y_ts})

      embedding_matrix = np.zeros((vocab_length, embed_dim))
      for word, index in word_tokenizer.word_index.items():
          embedding_vector = self.fasttext_embeddings_index.get(word)
          if embedding_vector is not None:
              embedding_matrix[index] = embedding_vector

      # print(x_train_padded)
      # print(x_test_padded)
      return ((tf.convert_to_tensor(x_train_padded,np.int32),tf.convert_to_tensor(dummy_y_tr,np.float32)),(tf.convert_to_tensor(x_test_padded,np.int32),tf.convert_to_tensor(dummy_y_ts,np.float32)),(vocab_length,embedding_matrix,embed_dim,length_long_sentence))


  def get_glove_preprocessing_params(self):
      return self.glove_wt,self.lls

  def get_fasttext_preprocessing_params(self):
      return self.fasttext_wt,self.lls

  def get_labels_df(self):
      return self.df_labels
  
  def data_per_fold_glove(self):
      # self.data_glove = {}
      embed_dim = 200
      self.data_glove = self.create_glove_embedding_matrix(embed_dim)

  def data_per_fold_fasttext(self):
      # self.data_fasttext = {}
      embed_dim = 300
      self.data_fasttext = self.create_fasttext_embedding_matrix(embed_dim)


  def process_glove(self):
      models = [self.CONV1D_BiLSTM]#self.SEP_CONV1D_BiRNN,self.SEP_CONV1D_BiGRU,self.BiGRU,self.BiLSTM,self.BiRNN]
      for mod in models:
          name = str(mod)
          name = name.replace('self.','')
          m,h = self.run_training(mod)
          self.models_glove[name] = m
          self.history_glove[name] = h
          # self.accuracy_glove['Test accuracy score for '+name] = acc

  def process_fasttext(self):
      models = [self.CONV1D]
      for mod in models:
          name = str(mod)
          name = name.replace('self.','')
          m,h = self.run_training(mod)
          self.models_fasttext[name] = m
          self.history_fasttext[name] = h
          # self.accuracy_glove['Test accuracy score for '+name] = acc
    
  def get_models_glove(self):
    return self.models_glove

  def get_history_glove(self):
    return self.history_glove

  def get_models_fasttext(self):
    return self.models_fasttext

  def get_history_fasttext(self):
    return self.history_fasttext


  def run_training(self,model):
      mod = []
      his = []
      x,h= model()
      mod.append(x)
      his.append(h)
    
      
      return mod,his

  def SEP_CONV1D(self):
      data = self.data_fasttext
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_100_SEP_CONV1D_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_SEP_CONV1D_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_SEP_CONV1D_embedding"),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(5, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      x = model.to_json()
      print("\n SEPERABLE CONV1D \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
      # y_pred = model.predict(x_ts)
      return x,his





  def CONV1D(self):
      data = self.data_glove
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_100_CONV1D_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_CONV1D_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_CONV1D_embedding"),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(5, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      x = model.to_json()
      print("\n CONV1D \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
      # y_pred = model.predict(x_ts)
      return model,his#,accuracy_score(y_ts,y_pred)

  def BiLSTM(self):
      data = self.data_glove
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_100_BiLSTM_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_BiLSTM_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_BiLSTM_embedding"),
              # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              # tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              # tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              # tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              # tf.keras.layers.Dense(5, activation='softmax')
              tf.keras.layers.Dense(2, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      x = model.to_json()
      print("\n BiLSTM \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
      # y_pred = model.predict(x_ts)
      return model,his#,accuracy_score(y_ts,y_pred)

  def SEP_CONV1D_BiLSTM(self):
      data = self.data_fasttext
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_100_SEP_CONV1D_BiLSTM_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_SEP_CONV1D_BiLSTM_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_SEP_CONV1D_BiLSTM_embedding"),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(5, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      x = model.to_json()
      print("\n SEPARABLE CONV1D BiLSTM \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
      # y_pred = model.predict(x_ts)
      return x,his #,accuracy_score(y_ts,y_pred)


  def CONV1D_BiLSTM(self):
      data = self.data_glove
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_100_CONV1D_BiLSTM_embedding"),
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_CONV1D_BiLSTM_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_CONV1D_BiLSTM_embedding"),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(5, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      x = model.to_json()
      print("\n CONV1D BiLSTM \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
      # y_pred = model.predict(x_ts)
      return model,his #,accuracy_score(y_ts,y_pred)


  def BiRNN(self):
      data = self.data_fasttext
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_100_BiRNN_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_BiRNN_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_BiRNN_embedding"),
              # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              # tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              # tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              # tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(5, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      x = model.to_json()
      print("\n BiRNN \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
      # y_pred = model.predict(x_ts)
      return x,his #,accuracy_score(y_ts,y_pred)

  def SEP_CONV1D_BiRNN(self):
      data = self.data_fasttext
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_100_SEP_CONV1D_BiRNN_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_SEP_CONV1D_BiRNN_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_SEP_CONV1D_BiRNN_embedding"),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(5, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      x = model.to_json()
      print("\n SEPARABLE CONV1D BiRNN \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
      # y_pred = model.predict(x_ts)
      return model,his #,accuracy_score(y_ts,y_pred)

  
  def CONV1D_BiRNN(self):
      data = self.data_fasttext
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_100_CONV1D_BiRNN_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_CONV1D_BiRNN_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_CONV1D_BiRNN_embedding"),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(5, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      x = model.to_json()
      print("\n CONV1D BiRNN \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
      # y_pred = model.predict(x_ts)
      return x,his #,accuracy_score(y_ts,y_pred)

  def BiGRU(self):
      data = self.data_fasttext
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_100_BiGRU_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_BiGRU_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_BiGRU_embedding"),
              # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              # tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              # tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              # tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(5, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      x = model.to_json()
      print("\n BiGRU \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
      # y_pred = model.predict(x_ts)
      return x,his #,accuracy_score(y_ts,y_pred)
      
  def CONV1D_BiGRU(self):
      data = self.data_glove
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_100_CONV1D_BiGRU_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_CONV1D_BiGRU_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_CONV1D_BiGRU_embedding"),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(2, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      x = model.to_json()
      print("\n CONV1D BiGRU \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
      # y_pred = model.predict(x_ts)
      return model,his #,accuracy_score(y_ts,y_pred)
      
  def SEP_CONV1D_BiGRU(self):
      data = self.data_fasttext
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_100_SEP_CONV1D_BiGRU_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_SEP_CONV1D_BiGRU_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_SEP_CONV1D_BiGRU_embedding"),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(5, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      x = model.to_json()
      print("\n SEPARABLE CONV1D BiGRU \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
      # y_pred = model.predict(x_ts)
      return x,his #,accuracy_score(y_ts,y_pred)

In [ ]:
wem = word_emb_model('/content/drive/MyDrive/sentiment data/tr_sen_fin.csv')
# wem = word_emb_model('/content/drive/MyDrive/sarcasm data/train_sarc_final_with_folds.csv')

Found 1193514 word vectors.


,u_id,text,label,kfold,lbl_num
0,30c09bb7414411eefe6dee09a33bdf15ae380d0a00e15a...,spent weekend being plagued by hayfever my sid...,sarcasm,0.0,1
1,44f44cc82c659219433181641314753aecff83a5e842ae...,oh no i had strep five times in a row once med...,sarcasm,0.0,1
2,6474df6d3bbbc4c5c61ef273dd2c7bd345c5ef12422697...,i wish we had something amazing to do for spri...,sarcasm,0.0,1
3,11903b6d411ab0790f7d82598ab6b4419c545628414a4d...,sometimes i just wished breakfast would cook i...,sarcasm,0.0,1
4,5afc61ff8dba0dd2066ef12716c075e1d6fec4ea8e1155...,recovering from my crazy family i love them bu...,sarcasm,0.0,1
...,...,...,...,...,...
293995,ffbffcc7e1e638709d88851edf89027a6521da93d8108e...,the ball does not touch the ground over the tr...,normal,4.0,0
293996,3361180b3bc4ef682f12371a79a4681df6b19257324f9f...,twitter seems empty today,normal,4.0,0
293997,90b2b11a0218559513d973697aeab27cb92fb5e83081fc...,that is evil,normal,4.0,0
293998,d50afb07f428fb519a1c85a1d84acc5f269360b1b7555a...,i love chasing pavements on my ipod too actual...,normal,4.0,0


In [ ]:
ft_prep_params = wem.get_fasttext_preprocessing_params()

In [ ]:
wem.process_glove()


 CONV1D BiLSTM 

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
GL_200_CONV1D_BiLSTM_embeddi (None, 163, 200)          27423200  
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 161, 32)           19232     
_________________________________________________________________
max_pooling1d_15 (MaxPooling (None, 80, 32)            0         
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 78, 32)            3104      
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, 39, 32)            0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 37, 32)            3104      
_________________________________________________________________
max_pooling1d_17 (MaxPooling (None, 

In [ ]:
# (<keras_preprocessing.text.Tokenizer at 0x7f324e88b0d0>, 163)
# (<keras_preprocessing.text.Tokenizer at 0x7f3237291510>, 163)
ft_prep_params

(<keras_preprocessing.text.Tokenizer at 0x7f3240ffbf10>, 163)

In [ ]:
import pickle

with open('/content/drive/MyDrive/sarcasm models/ft_300_prep_params.pkl', 'wb') as file:
    pickle.dump(ft_prep_params, file)

In [ ]:
# d = {'VNeg':0,'Neg':1,'Neu':2,'Pos':3,'VPos':4}
d = {'sarcasm':1,'normal':0}
df['lbl_num'] = df['label'].apply(lambda x: d[x])

In [ ]:
df.to_csv('/content/drive/MyDrive/sarcasm data/train_sarc_final_with_folds.csv',index = False,header = True)

In [ ]:
x = wem.get_models_glove()
print(x)

{'<bound method word_emb_model.CONV1D_BiLSTM of <__main__.word_emb_model object at 0x7f7a194e8f50>>': [<tensorflow.python.keras.engine.sequential.Sequential object at 0x7f7a21818290>]}


In [ ]:
mod = x['<bound method word_emb_model.CONV1D_BiLSTM of <__main__.word_emb_model object at 0x7f7a194e8f50>>'][0]

In [ ]:
k = wem.create_fasttext_embedding_matrix(300)

In [ ]:
x_tr,y_tr = k[0]
x_ts,y_ts = k[1]

In [ ]:
# ypred = mod.predict_classes(x_ts)
y_pred =(mod.predict(x_ts) > 0.5).astype("float32")

In [ ]:
labels_pred = np.argmax(y_pred, 1)

In [ ]:
from sklearn.metrics import accuracy_score
z = accuracy_score(y_ts,y_pred)

In [ ]:
y_classes = np.array([np.argmax(y, axis=None, out=None) for y in (mod_l.predict(x_ts) > 0.5).astype("float32")])

In [ ]:
np.unique(y_classes)

array([0, 1, 2, 3, 4])

In [ ]:
from collections import Counter
c = Counter(list(ypred))

In [ ]:
# c1 = Counter(
y_ts

<tf.Tensor: shape=(138000, 5), dtype=float32, numpy=
array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)>

In [ ]:
mod.save('/content/drive/MyDrive/sentiment models/GLOVE_200D_CONV1D_BiLSTM.h5')

In [ ]:
from keras.models import load_model
mod_l = load_model('/content/drive/MyDrive/sarcasm models/GLOVE_100D_CONV1D_BiGRU.h5')

In [ ]:
mod_l.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 163, 100)          8445700   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 161, 32)           9632      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 80, 32)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 78, 32)            3104      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 39, 32)            0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 37, 32)            3104      
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 18, 32)           

In [ ]:
# import pickle
y = wem.get_history_fasttext()
# with open('/content/drive/MyDrive/sent_fasttext_300_acc.pickle', 'wb') as handle:
#     pickle.dump(y, handle, protocol=pickle.HIGHEST_PROTOCOL)






# z = y['<bound method word_emb_model.CONV1D of <__main__.word_emb_model object at 0x7fa8481759d0>>'][0].history

In [ ]:
k = ['SEPARABLE CONV1D BiRNN','SEPARABLE CONV1D BiGRU','BiGRU','BiLSTM','BiRNN']

In [ ]:
dfs = []
for k in y.keys():
  z = y[k][0].history
  df = pd.DataFrame.from_dict(z)
  df['epoch'] = [i+1 for i in range(len(df))]
  dfs.append(df)

In [ ]:
len(dfs)

5

In [ ]:
for i in y.keys():
  print(i)

<bound method word_emb_model.SEP_CONV1D_BiRNN of <__main__.word_emb_model object at 0x7f690e05b7d0>>
<bound method word_emb_model.SEP_CONV1D_BiGRU of <__main__.word_emb_model object at 0x7f690e05b7d0>>
<bound method word_emb_model.BiGRU of <__main__.word_emb_model object at 0x7f690e05b7d0>>
<bound method word_emb_model.BiLSTM of <__main__.word_emb_model object at 0x7f690e05b7d0>>
<bound method word_emb_model.BiRNN of <__main__.word_emb_model object at 0x7f690e05b7d0>>


In [ ]:
k = ['CONV1D','SEP_CONV1D','CONV1D_BiLSTM','CONV1D_BiRNN','CONV1D_BiGRU']

In [ ]:
for i in range(len(dfs)):
  dfs[i].to_csv('/content/drive/MyDrive/sent_fasttext_300_'+k[i]+'.csv',index = False,header = True)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/sarcasm data/train_sarc_final.csv')

In [ ]:
def check(x):
  if type(x) == str:
    return True
  return False

In [ ]:
df['inv'] = df['text'].apply(lambda x: check(x))

In [ ]:
df = df.drop('inv',axis = 1)

In [ ]:
num_words = [len(s.split()) for s in df['preprocessed']]
# df.columns

In [ ]:
val_ds[1]

NameError: ignored